# Setup Scratch

In [7]:
import os

# scratch directory is apart of the .gitignore to ensure it is not committed to git
%env SCRATCH=../scratch
! [ -e "${SCRATCH}" ] || mkdir -p "${SCRATCH}"

scratch_path = os.environ.get('SCRATCH', 'scratch')

env: SCRATCH=../scratch


In [8]:
# fetch all env variable these values will be fetched from Data Connection setup

access_key = os.getenv("AWS_ACCESS_KEY_ID")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
s3_endpoint = os.getenv("AWS_S3_ENDPOINT").lstrip('http://')
bucket_name = os.getenv("AWS_S3_BUCKET")

In [14]:
# create s3 bucket
# upload files to bucket
# download files from bucket

#!/usr/bin/python3
from minio import Minio
import sys
import os
import urllib3
import glob

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Create a Minio client object
minio_client = Minio(s3_endpoint,
                     access_key=access_key,
                     secret_key=secret_key,
                     secure=False)

def upload_local_directory_to_minio(local_path, bucket_name):
    assert os.path.isdir(local_path)

    # Upload files in the directory to the bucket
    for file_name in glob.glob(local_path + '/**'):
        file_name = file_name.replace(os.sep, "/")
        print("file_name: " + file_name)
        if not os.path.isfile(file_name):
            upload_local_directory_to_minio(
                file_name, bucket_name)
        else:
            remote_path = os.path.join(
                local_path.lstrip('../scratch/'), file_name[1 + len(file_name):])
            remote_path = remote_path.replace(os.sep, '/')
            print("remote_path: " + remote_path, file_name)
            minio_client.fput_object(bucket_name, remote_path, os.path.basename(file_name))

try:
    # Check if the bucket already exists
    if not minio_client.bucket_exists(bucket_name):
        minio_client.make_bucket(bucket_name)
        print(f"Bucket '{bucket_name}' created successfully.")

    upload_local_directory_to_minio(scratch_path, bucket_name)

except Exception as err:
    print(f"Failed to upload files to bucket '{bucket_name}': {err}")

file_name: ../scratch/Vegetable Images
file_name: ../scratch/Vegetable Images/test
file_name: ../scratch/Vegetable Images/test/Radish
file_name: ../scratch/Vegetable Images/test/Radish/1044.jpg
remote_path: Vegetable Images/test/Radish/ ../scratch/Vegetable Images/test/Radish/1044.jpg
Failed to upload files to bucket 'models': [Errno 2] No such file or directory: '1044.jpg'


In [5]:

s3 = boto3.resource(
    "s3",
    aws_access_key_id=access_key,
    aws_secret_access_key=secret,
    config=boto3.session.Config(signature_version="s3v4"),
    verify=False,
    endpoint_url=s3_endpoint,
)
s3_client = s3.meta.client

bucket = s3.Bucket(bucket_name)

objects = bucket.objects.filter(Prefix="/")
for obj in objects:
    path, filename = os.path.split(obj.key)
    if len(path) != 0 and not os.path.exists(os.path.join(local_dest_dir, path)):
        os.makedirs(os.path.join(local_dest_dir, path))
    if not obj.key.endswith("/"):
        download_to = local_dest_dir + path + "/" + filename if path else filename
        s3_client.download_file(bucket_name, obj.key, download_to)

NameError: name 'boto3' is not defined